**Libraries**

In [1]:
import pandas as pd
import sqlite3

**Screapping municipios and departamentos codes**

In [2]:
url = 'https://datos.gov.co/api/views/gdxc-w37w/rows.csv?accessType=DOWNLOAD&bom=true&format=true&delimiter=%3B'
municipios = pd.read_csv(url, sep= ';', dtype = str)[0:1121]

In [3]:
municipios.columns = ['cod_depto', 'cod_mpio', 'dpto', 'nom_mpio', 'tipo']

**Connecting to database**

In [4]:
con = sqlite3.connect('Colombia_Demographic_data.sqlite')
cur = con.cursor()

**Insert department**

In [5]:
insert_depto = 'INSERT INTO Departamento(id, name) VALUES("00", "Nacional"),'

insert_depto += '('

for cod_depto in municipios['cod_depto'].unique():
    dept = municipios['dpto'][municipios['cod_depto'] == cod_depto].unique()
    insert_depto += '"'+str(cod_depto) + '"'+ ',' + '"'+str(dept[0].capitalize()) + '"' + '),('
insert_depto = insert_depto[:-2] +';'

In [6]:
cur.execute(insert_depto)
con.commit()

**Insert municipio type**

In [7]:
insert_tipo_mpio = 'INSERT INTO Tipo_municipio(name) VALUES'

insert_tipo_mpio += '('

for tipo in municipios['tipo'].unique():
    insert_tipo_mpio += '"'+str(tipo) + '"' + '),('
    
insert_tipo_mpio = insert_tipo_mpio[:-2] +';'

In [8]:
cur.execute(insert_tipo_mpio)
con.commit()

**Insert municipio**

In [9]:
insert_mpio = 'INSERT INTO Municipio(id, name, departamento_id, tipo_mpio_id) VALUES'

insert_mpio += '('

for cod_mpio in municipios['cod_mpio'].unique():
    mpio = municipios['nom_mpio'][municipios['cod_mpio'] == cod_mpio].unique()
    dept = municipios['cod_depto'][municipios['cod_mpio'] == cod_mpio].unique()
    tipo = municipios['tipo'][municipios['cod_mpio'] == cod_mpio].unique()[0]
    
    select_query = 'SELECT id FROM '
    select_query += 'Tipo_municipio' + ' WHERE name = ? ;'
    cur.execute(select_query, (tipo, ))
    
    try:
        type_id = cur.fetchone()[0]
    except:
        type_id = 'NULL'
        
    insert_mpio += '"'+str(cod_mpio) + '"'+ ',' + '"'+str(mpio[0].capitalize()) + '"'+ ',' +'"'+ str(dept[0].capitalize()) + '"' + ',' +'"'+ str(type_id) + '"''),('
insert_mpio = insert_mpio[:-2] +';'

In [10]:
cur.execute(insert_mpio)
con.commit()

**Disconnecting to the database**

In [11]:
con.close()